In [1]:
import json

with open("../data/trace_dissection/benign.json") as f:
    benign_data = json.load(f)

def is_float(value) -> bool:
    try:
        float(value)
        return True
    except ValueError:
        return False

list_identifiers = [
    "ip_src"
]

float_encountered = {}
word_encountered  = {}

for packet in benign_data:

    for key, value in packet["http2"].items():

        if key not in list_identifiers:


            # Integers
            if is_float(value):
                if key not in float_encountered:
                    float_encountered[key] = []
                float_encountered[key].append(float(value))

            # Text
            else :
                if key not in word_encountered:
                    word_encountered[key] = []
                word_encountered[key].append(value)

In [ ]:
import json
import re

ban_list = [
    "path",
    "jwt.sub",
    "InstanceId",
    "ipv4Address",
    "notifyUri",
    "notificationUri",
    "Timestamp",
    "smContextStatusUri",
    "kseaf",
    "resStar",
    "authenticationVector",
    "5gAuthData",
    "kausf"
]
word_encountered_sets = {}
for key, value in word_encountered.items():

    key = re.sub(r'\[\d+\]', '', key)

    found = False
    for ban_word in ban_list:
        if ban_word.lower() in key.lower():
            found = True

    if not found:
        if key not in word_encountered_sets:
            word_encountered_sets[key] = []
        word_encountered_sets[key] = list(set(value + word_encountered_sets[key]))

keys_tot = list(word_encountered_sets.keys())
values_tot = []
for key, values in word_encountered_sets.items():
    values_tot += values

with open("word_encountered_unique.json", "w") as f:
    json.dump(word_encountered_sets, f, indent=2)

In [26]:
len(set(values_tot)) + len(set(keys_tot))

3490

In [ ]:
a = [
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/npcf-smpolicycontrol/v1/sm-policies",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/guami",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-disc/v1/nf-instances",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/sequenceNumber",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nausf-auth/v1/ue-authentications",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nchf-convergedcharging/v3/chargingdata",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/oauth2/token",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnssf-nsselection/v2/network-slice-information",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/npcf-am-policy-control/v1/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify",
    "/nnrf-nfm/v1/nf-instances/NF_ID",
    "/nnrf-nfm/v1/nf-instances/NF_ID"
  ],
set(a[0])

{'/guami',
 '/nausf-auth/v1/ue-authentications',
 '/nchf-convergedcharging/v3/chargingdata',
 '/nnrf-disc/v1/nf-instances',
 '/nnrf-nfm/v1/nf-instances/NF_ID',
 '/nnssf-nsselection/v2/network-slice-information',
 '/npcf-am-policy-control/v1/SUB',
 '/npcf-smpolicycontrol/v1/sm-policies',
 '/nsmf-pdusession/v1/sm-contexts',
 '/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/modify',
 '/nsmf-pdusession/v1/sm-contexts/urn:uuid:NF_ID/release',
 '/nudr-dr/v2/application-data/influenceData',
 '/nudr-dr/v2/application-data/influenceData/subs-to-notify',
 '/nudr-dr/v2/application-data/influenceData/subs-to-notify/SUB',
 '/oauth2/token',
 '/sequenceNumber'}

: 

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch_geometric.utils import from_networkx

# Create a simple graph
G = nx.Graph()

# Store embeddings as lists (numeric)
G.add_node("A", embedding=1)
G.add_node("B", embedding=2)
G.add_node("C", embedding=3)

G.add_edge("A", "B", embedding=4)
G.add_edge("B", "C", embedding=4)
G.add_edge("C", "A", embedding=4)

# Convert to PyG Data object
b = from_networkx(G, group_node_attrs=["embedding"], group_edge_attrs=["embedding"])

print(type(b))

<class 'torch_geometric.data.data.Data'>


AttributeError: 'list' object has no attribute 'dim'

In [14]:
b

Data(edge_index=[2, 6], num_nodes=3)

In [10]:
import numpy as np

node_embeddings = np.array([G.nodes[n]['embedding'] for n in G.nodes()])
edge_embeddings = np.array([G.edges[edge]['embedding'] for edge in G.edges()])

In [11]:
node_embeddings

array(['1', '2', '3'], dtype='<U1')

3500 (avec id) vs 320 (sans id)